In [1]:
from working_with_files import *
from preprocessing import *
import pandas as pd
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [2]:
def sort_by_official_dates(df):
    df1 = pd.DataFrame() # < 2008
    df2 = pd.DataFrame() # 2008 - 2013
    df3 = pd.DataFrame() # 2013 - 2016
    df4 = pd.DataFrame() # > 2016
    date1 = datetime.strptime('15-07-2008', '%d-%m-%Y').date()
    date2 = datetime.strptime('20-02-2013', '%d-%m-%Y').date()
    date3 = datetime.strptime('30-11-2016', '%d-%m-%Y').date()
    for index, row in df.iterrows():
        if int(row['Unnamed: 0']) not in [10995, 11494]:
            try:
                date = datetime.strptime(row['date'].strip(), '%d-%m-%Y').date()
            except ValueError:
                date = datetime.strptime(row['date'], '%m-%Y').date()
            if date <= date1:
                df1 = df1.append(row, ignore_index=True)
            if date > date1 and date <= date2:
                df2 = df2.append(row, ignore_index=True)
            if date > date2 and date <= date3:
                df3 = df3.append(row, ignore_index=True)
            if date > date3:
                df4 = df4.append(row, ignore_index=True)
    return df1, df2, df3, df4

def show_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print('Topic {}:'.format(topic_idx))
        print(' '.join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))
    print()

def get_topics(documents, n_topics, n_top_words, models):
    
    random_seed = 42
    
    if 'nmf' in models:
        tfidf = tfidf_vectorizer.fit_transform(documents)
        tfidf_feature_names = tfidf_vectorizer.get_feature_names()
        
        nmf = NMF(n_components=n_topics, random_state=random_seed, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)
        
        print('NMF')
        show_topics(nmf, tfidf_feature_names, n_top_words)
        
    if 'lda' in models:
        tf = tf_vectorizer.fit_transform(documents)
        tf_feature_names = tf_vectorizer.get_feature_names()
        
        lda = LatentDirichletAllocation(n_components=n_topics, max_iter=5, learning_method='online', learning_offset=50., 
                                random_state=random_seed).fit(tf)
        
        print('LDA')
        show_topics(lda, tf_feature_names, n_top_words)

In [3]:
# read df and split it into 4 separate df4 based on publication date
df = pd.read_csv('mfa_texts_df.csv', sep = '\t', encoding = 'utf-8')
sorted_dfs = sort_by_official_dates(df)

In [4]:
# statistics 
n = 0
for df in sorted_dfs:
    print('Corpus {}:'.format(n), len(df), 'texts', len(get_all_texts(df).split()), 'words')
    n += 1

Corpus 0: 1583 texts 845988 words
Corpus 1: 3168 texts 1111520 words
Corpus 2: 3344 texts 1082001 words
Corpus 3: 4277 texts 1420393 words


In [4]:
# get unnecessary symbols to remove them from texts
symbols_to_del = generate_symbols_to_del(get_all_texts(df))
symbols_to_del.extend(list('abcdefghijklmnopqrstuvwxyz'))
print(symbols_to_del)

['"', '´', '8', '4', '`', '&', '−', '=', '„', '%', '！', ']', '?', '－', '$', '3', '1', '0', '…', '\xad', '€', '+', ',', '.', '@', '№', '’', ':', '―', '‒', '>', '\n', '«', '*', '#', '(', '·', '“', '”', '[', '_', '/', '5', '2', '‑', '6', '{', '–', '»', ')', '<', '9', '́', '°', '!', ';', '—', '7', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [5]:
# clean texts from unnecessary symbols

for df in sorted_dfs:
    df['text_clean'] = df['text_lemmatized'].apply(clean, args=(symbols_to_del,))

In [159]:
# get and preprocess stopwords
# source: ruscorpora.ru

stopwords_raw = get_lexicon_from_file('russian_stopwords.txt')
stopwords = []
stopwords_string = ''.join(stopwords_raw)
strings = stopwords_string.split('\t')
for string in strings:
    if string.isalpha():
        stopwords.append(string)

# lemmatize stopwords

stopwords_lemmatized = [lemmatize_pymorphy(word) for word in stopwords]
stopwords_lemmatized = list(set(stopwords_lemmatized))
stopwords = [stopword for stopword in stopwords_lemmatized if stopword.isalpha()]

# stopwords.extend(['наш', 'твой', 'мой', 'свой', 'ваш', 'его', 'их', 'ее', 'над', 'чей', 'между', 'год', 'вопрос', 'страна', 
 #                'россия', 'российский', 'такой', 'государство', 'другой', 'также', 'рф', 'мид', 'сегодня', 'два', 'катерина',
 #                'спецкор', 'дать', 'прокомментировать', 'интервью', 'федерация', 'лавров', 'сторона', 'российско', 'господин',
 #                'рассказывать', 'министр', 'встреча', 'состояться', 'сайт', 'итог', 'официальный', 'глава', 'иностранный',
 #                'дело', 'москва', 'александр', 'курировать', 'иванов', 'делегации', 'заместитель', 'комментировать', 'уважаемый',
 #                'президент', 'москве', 'государственный' 'лавр', 'москвы', 'принять', 'участие', 'мероприятие', 'россии', 
 #                 'медведев', 'пресс', 'посетить'])

stopwords.extend('''б г д е ж з й л м н п р т ф х ц ч ш щ ъ ы ь э ю
                наш твой мой свой ваш его их ее над чей такой другой между год страна государство также все отношение г-н 
                россия российский сторона де два дж мид внешнеполитический интервью министр газета иностранный дело
                катерина спецкор новость рассказывать подобный кроме министерство кстати пресс-конференция москва сайт лавров
                медведев путин сайт подробный рф s форум сердечно добрый желать приветствовать сам ясный ясность кто-то
                говорить хотеть официальный еще какой-то об лишь включая вокруг ле'''.split())

print(stopwords)

['то', 'она', 'мы', 'из', 'что', 'бы', 'и', 'себя', 'чтобы', 'очень', 'но', 'быть', 'как', 'весь', 'вот', 'ты', 'по', 'здесь', 'потому', 'это', 'за', 'можно', 'а', 'ничто', 'этот', 'или', 'мочь', 'под', 'потом', 'до', 'нет', 'при', 'человек', 'они', 'вы', 'же', 'сказать', 'есть', 'не', 'с', 'где', 'только', 'о', 'ну', 'я', 'который', 'через', 'ни', 'более', 'от', 'он', 'уже', 'после', 'для', 'там', 'в', 'на', 'у', 'без', 'теперь', 'кто', 'так', 'раз', 'тот', 'ли', 'к', 'когда', 'если', 'время', 'тем', 'чем', 'надо', 'один', 'даже', 'ещё', 'тут', 'тогда', 'б', 'г', 'д', 'е', 'ж', 'з', 'й', 'л', 'м', 'н', 'п', 'р', 'т', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'наш', 'твой', 'мой', 'свой', 'ваш', 'его', 'их', 'ее', 'над', 'чей', 'такой', 'другой', 'между', 'год', 'страна', 'государство', 'также', 'все', 'отношение', 'г-н', 'россия', 'российский', 'сторона', 'де', 'два', 'дж', 'мид', 'внешнеполитический', 'интервью', 'министр', 'газета', 'иностранный', 'дело', 'катерина', 'с

In [160]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.9, min_df=10, stop_words=stopwords, token_pattern=r"\b['\w-]+\b")
tf_vectorizer = CountVectorizer(max_df=0.9, min_df=10, stop_words=stopwords, token_pattern=r"\b['\w-]+\b")

In [161]:
get_topics(sorted_dfs[0].loc[:, 'text_clean'], 38, 15, ['nmf']) # до 2008

NMF
Topic 0:
международный развитие новый мир мировой политика проблема решение становиться интерес сотрудничество важный экономический глобальный регион
Topic 1:
израиль палестинец дорожный карта израильский палестино-израильский четверка посредник квартет урегулирование палестинский израильтянин пна ближневосточный аббас
Topic 2:
сотрудничество вопрос обсуждать переговоры визит президент договариваться двусторонний встреча взаимодействие договоренность сегодня торговый-экономический сфера коллега
Topic 3:
палестинский пна аббас хамас территория администрация межпалестинский израиль глава правительство палестинец национальный народ израильский обстановка
Topic 4:
европа совет се европейский председательство конвенция комитет парламентский правовой общеевропейский право ассамблея разделительный организация пасе
Topic 5:
церковь православный русский святой общество религиозный патриарх духовный цивилизация религия земля дипломатия конфессия мир единство
Topic 6:
соотечественник рубеж ди

In [162]:
get_topics(sorted_dfs[1].loc[:, 'text_clean'], 39, 15, ['nmf']) # 2008 - 2013

NMF
Topic 0:
международный мир развитие новый глобальный мировой становиться работа безопасность интерес основа политика решение система проблема
Topic 1:
осетия южный абхазия республика закавказье женевский граница абхазский грузия неприменение дискуссия югоосетинский сила соглашение цхинвал
Topic 2:
сотрудничество двусторонний взаимодействие обсуждать переговоры вопрос сфера область коллега торговый-экономический связь совместный ход развитие сегодня
Topic 3:
соотечественник русский рубеж язык конференция зарубежный конгресс проживать всемирный правительственный диаспора община координационный комиссия работа
Topic 4:
гражданин посольство консульский власть информация сотрудник ситуация виза находиться россиянин необходимый сми связь поступать погибший
Topic 5:
сша американский российский-американский вашингтон про обама клинтон госдепартамент вопрос госсекретарь администрация президент штат снв государственный
Topic 6:
снг содружество заседание глава совет государство-участник казах

In [163]:
get_topics(sorted_dfs[2].loc[:, 'text_clean'], 38, 15, ['nmf']) # 2013 - 2016

NMF
Topic 0:
международный развитие сотрудничество регион интерес работа экономический новый сегодня усилие мир безопасность проблема основа друг
Topic 1:
сирийский сирия оппозиция боевик террорист алеппо сара дамаск вооруженный группировка джабхат правительство женевский группа район
Topic 2:
украина украинский киев власть киевский юго-восток минский заявление ситуация донецкий донбасс действие донецк февраль договоренность
Topic 3:
визит двусторонний сотрудничество федерация переговоры встреча вопрос обсуждать ход актуальный состояться республика взаимодействие повестка торговый-экономический
Topic 4:
погибший выздоровление соболезнование пострадавший осуждать решительно террористический погибать выражать близкие скорый теракт родные ранение результат
Topic 5:
сша американский вашингтон администрация санкция против госдепартамент российский-американский керри про обама заявление белый санкционный госсекретарь
Topic 6:
оон резолюция сб совет безопасность генеральный право международны

In [167]:
get_topics(sorted_dfs[3].loc[:, 'text_clean'], 39, 15, ['nmf']) # С 2016

NMF
Topic 0:
международный интерес сотрудничество регион безопасность развитие работа экономический проблема мир число важный новый решение основа
Topic 1:
сша американский вашингтон американец договор санкция госдепартамент администрация военный против ракета дрсмд вооружение заявление никакой
Topic 2:
сирийский сирия турция астанинский процесс урегулирование оон резолюция иран конгресс сб сочи оппозиция астана сириец
Topic 3:
украина украинский киев киевский донбасс власть минский закон режим порошенко миссия действие нынешний очередной вышинский
Topic 4:
визит двусторонний сотрудничество переговоры федерация республика актуальный обсуждать региональный вопрос рабочий повестка состояться ход взаимодействие
Topic 5:
мероприятие молодой участие дипломат федерация представитель дипломатический принимать состояться совет федеральный приглашать встреча посвящать международный
Topic 6:
сирия сирийский террорист боевик провинция гуманитарный район зона житель лагерь пункт военный помощь але